In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta

# Imports and feature selection

In [3]:
# using docs to model
# labels come from claims data 
claims = pd.read_csv('/Users/drewrice/Desktop/capson_csv/Claims.csv')
docs = pd.read_csv('/Users/drewrice/Desktop/capson_csv/ProviderInfo.csv')
prems = pd.read_csv('/Users/drewrice/Desktop/capson_csv/ScheduleOfWrittenPremium.csv')
debs_and_creds = pd.read_csv('/Users/drewrice/Desktop/capson_csv/ScheduleOfDebitsAndCredits.csv')

In [4]:
# drop
docs.drop(['Middle Name','Address 1', 'Address 2', 'Zip','Address Type','Practice Start Date'],axis=1,inplace=True)
docs.drop_duplicates(inplace=True)

In [5]:
indem = claims[['Indemnity_Paid','ProviderId']]

In [6]:
# docs DF, conversion to string
# lower ProviderId
docs[['ProviderId','Last Name','First Name','Gender','City','State','County']] = \
docs[['ProviderId','Last Name','First Name','Gender','City','State','County']].astype(str)
docs['ProviderId'] = docs['ProviderId'].map(lambda x: x if type(x)!=str else x.lower())

In [7]:
# indem DF, conversions to string and float
# lower ProviderId
indem['Indemnity_Paid'] = indem['Indemnity_Paid'].str.replace(r'[$,]', '').astype('float')
indem['ProviderId'] = indem['ProviderId'].astype(str)
indem['ProviderId'] = indem['ProviderId'].str.lower()

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.h

# Join docs and indemnity tables

In [8]:
joined = pd.merge(docs, indem, how='outer', on='ProviderId').merge()
joined.drop_duplicates(inplace=True)

In [34]:
#joined[joined['Indemnity_Paid'] == 0].sort(['Last Name'])

# Feature engineering

In [10]:
# build age column
joined['Date of Birth'] = pd.to_datetime(joined['Date of Birth'])

In [11]:
# time_delta function
def time_delta(dt):
    if dt.year > 2000:
        dt = dt - timedelta(days=36525)
    return dt

In [12]:
# apply time_delta function to DoB column, deal with datetime rounding to the 2000s
joined['Date of Birth'] = joined['Date of Birth'].apply(lambda dt: time_delta(dt))

In [13]:
# fill missing values with 1/1/2060
joined['Date of Birth'].fillna(datetime(2060, 1, 1, 0, 0),inplace=True)

In [14]:
# build age column
now = datetime.now()
joined['age'] = now - joined['Date of Birth']

In [16]:
def to_years(dt):
    if dt:
        dt = int(dt.strftime("%y"))
    return dt

In [17]:
# TODO: convert days to years in joined['age']
# TODO: drop 1/1/2060 instances
joined['age'] = joined['age'].apply(lambda dt: to_years(dt))

AttributeError: 'Timedelta' object has no attribute 'strftime'

# EDA

In [24]:
# resave debs_and_creds with just the desired columns
debs_and_creds = debs_and_creds[['ProviderId','BoardCredit','ClaimsBand']]

In [27]:
# group by Provider ID, resave debs_and_creds
debs_and_creds = debs_and_creds.groupby(debs_and_creds['ProviderId']).sum()

In [29]:
def build_classifier(item):
    if item == 0:
        return 0
    if item != 0:
        return 1

In [32]:
# use build_classifier to 
debs_and_creds['board_cert'] = debs_and_creds['BoardCredit'].apply(lambda item: build_classifier(item))
debs_and_creds['claims_band'] = debs_and_creds['ClaimsBand'].apply(lambda item: build_classifier(item))

In [33]:
debs_and_creds

,BoardCredit,ClaimsBand,board_cert,claims_band
ProviderId,,,,
00048B46-108A-4027-9D9C-C28C1BA8FFBF,0,-5844,0,1
000C64C1-1426-42FB-9E15-B9DE93ADDDE8,0,-4273,0,1
0014FA4E-F5F8-410A-875D-A707DE872718,-70,0,1,0
0019ADD8-FA3E-4605-B717-8B298CFFC7CF,0,0,0,0
00214018-3516-4F36-932C-A59BF34C81C9,-26,-129,1,1
0021B180-CC67-48D3-B2EE-9E57BE26BD55,-332,-1273,1,1
00465F57-9360-4673-AB95-87423C8B6060,0,-412,0,1
0069A526-3C93-4E97-9D93-6F3A7246DD22,0,0,0,0
00A0E0E1-589A-464C-B54B-D2A7FE65985E,0,0,0,0
